In [1]:
import pandas as pd
import numpy as np
import json
from PIL import Image, ImageDraw, ImageFont
import os
import regex

In [114]:
def preprocess_text(text):
    text = text.replace('\'\'', '').lower()
    split_text = regex.findall(r'\':?(.*?):?\'', text)
    return split_text

def split_emojis(s):
    return regex.findall(r"\X", s)

def preprocess_emoji(emojis):
    emojis_list = split_emojis(emojis)
    filtered = [c for c in emojis_list if c.strip() != '']
    return filtered

elco_df = pd.read_csv('../../data/elco.csv')
descs = elco_df["Description"]
emoji_dict = {}

for i, row in elco_df.iterrows():
    descs = preprocess_text(row["Description"])
    emojis = preprocess_emoji(row["EM"])
    if len(descs) != len(emojis):
        print(f"Error at: {i}, {descs}, {emojis}")
        break
    else:
        for j in range(len(descs)):
            emoji_dict[descs[j]] = emojis[j]
            
emoji_desc_pair = {pair[0]: pair[1] for pair in set(emoji_dict.items())}

In [117]:
def emoji_to_image(em, desc, font_path, image_size=128, save_dir='images'):
    os.makedirs(save_dir, exist_ok=True)
    image = Image.new("RGB", (image_size, image_size), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    font = ImageFont.truetype(font_path, 109)
    draw.text((0, 0), em, font=font, embedded_color=True)
    image.save(os.path.join(save_dir, f"{desc}.png"))


In [109]:
for desc, em in emoji_desc_pair.items():
    emoji_to_image(em, desc, '../../NotoColorEmoji.ttf')